In [335]:
!pip install category_encoders==2.*

In [336]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
#from pandas_profiling import ProfileReport
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, validation_curve # k-fold CV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # Hyperparameter tuning

In [490]:
def wrangle(fm_path, tv_path=None):
    if tv_path:
        df = pd.merge(pd.read_csv(fm_path,
                                  parse_dates=['date_recorded','construction_year'],
                                  na_values=[0, -2.000000e-08]),
                      pd.read_csv(tv_path))
        df.set_index('id')

        extra_elevations = pd.read_csv('Extra_Elevations - Extra_Elevations.csv')
        extra_elevations.columns = ['id','gps_height']
        df = df.merge(extra_elevations[['id', 'gps_height']], on='id', how='left')
        gps_height_sum = df['gps_height_x'].fillna(0) + df['gps_height_y'].fillna(0)
        df['gps_height_sum'] = gps_height_sum
        df['gps_height_sum'] = df['gps_height_sum'].replace(0,np.nan)

        df.drop(columns= ['gps_height_x','gps_height_y'], inplace=True)

        #drop duplicate observations (rows)
        df.drop_duplicates(inplace=True)  
        
    else:
        df = pd.read_csv(fm_path,
                         parse_dates=['date_recorded','construction_year'], 
                         na_values=[0, -2.000000e-08])
        df = df.set_index('id')
        
        
        extra_elevations = pd.read_csv('X_extra_X - X_extra_X.csv')
        extra_elevations.columns = ['id','gps_height']
        df = df.merge(extra_elevations[['id', 'gps_height']], on='id', how='left')
        gps_height_sum = df['gps_height_x'].fillna(0) + df['gps_height_y'].fillna(0)
        df['gps_height_sum'] = gps_height_sum
        df['gps_height_sum'] = df['gps_height_sum'].replace(0,np.nan)

        df.drop(columns= ['gps_height_x','gps_height_y'], inplace=True)
        
    # Drop constant columns
    df.drop(columns=['recorded_by'], inplace=True)
        
    # Drop duplicate columns
    dupe_cols = [col for col in df.head(100).T.duplicated().index
                 if df.head(100).T.duplicated()[col]]
    df.drop(columns=dupe_cols, inplace=True)    
    
    # Cleaning Extraction Type
    swn_mask = df['extraction_type'].str.contains('swn')
    df.loc[swn_mask,'extraction_type'] = 'swn'

    india_mask = df['extraction_type'].str.contains('india mark')
    df.loc[india_mask,'extraction_type'] = 'india mark'

    rope_mask = df['extraction_type'].str.contains('rope pump')
    df.loc[rope_mask,'extraction_type'] = 'rope pump'

    Ex_types = df['extraction_type'].value_counts()[:11].index

    Other_mask = df['extraction_type'].isin(Ex_types)
    df.loc[~Other_mask,'extraction_type'] = 'other'

    df.drop(columns=['extraction_type_group','extraction_type_class'], inplace=True)
    
    
    # Cleaning source
    Other_mask = df['source'].str.contains('unknown')
    df.loc[Other_mask,'source'] = 'other'

    df.drop(columns=['source_type','source_class'], inplace=True)
    
    # Cleaning Payment, Waterpoint, Water_Quality
    df.drop(columns=['payment','waterpoint_type_group','quality_group'], inplace=True)

    
    # Cleaning Founders
    df['funder'] = df['funder'].str.lower().fillna('other')

    mask = df['funder'].str.contains('private')
    df.loc[mask,'funder'] = 'private'

    mask = df['funder'].str.contains('government')
    df.loc[mask,'funder'] = 'government of tanzania'

    mask = df['funder'].str.contains('danida')
    df.loc[mask,'funder'] = 'danida'

    mask = df['funder'].str.contains('hesawa')
    df.loc[mask,'funder'] = 'hesawa'

    mask = df['funder'].str.contains('rwssp')
    df.loc[mask,'funder'] = 'rwssp'

    mask = df['funder'].str.contains('bank')
    df.loc[mask,'funder'] = 'bank'

    mask = df['funder'].str.contains('ministry')
    df.loc[mask,'funder'] = 'ministry'

    mask = df['funder'].str.contains('germany')
    df.loc[mask,'funder'] = 'germany'

    mask = df['funder'].str.contains('church')
    df.loc[mask,'funder'] = 'church'

    mask = df['funder'].str.contains('unicef')
    df.loc[mask,'funder'] = 'unicef'

    mask = df['funder'].str.contains('council')
    df.loc[mask,'funder'] = 'council'

    mask = df['funder'].str.contains('rural water')
    df.loc[mask,'funder'] = 'rwssp'

    mask = df['funder'].str.contains('water')
    df.loc[mask,'funder'] = 'water'

    mask = df['funder'].str.contains('gov')
    df.loc[mask,'funder'] = 'government of tanzania'

    mask = df['funder'].str.contains('kkk')
    df.loc[mask,'funder'] = 'kkkt'
    
    founders = df['funder'].value_counts()[:11].index
    mask = df['funder'].isin(founders)
    df.loc[~mask,'funder'] = 'other'
    
    df['installer'] = df['installer'].str.lower().fillna('other')

    
    #Cleaning Installers
    mask = df['installer'].str.contains('private')
    df.loc[mask,'installer'] = 'private'

    mask = df['installer'].str.contains('government')
    df.loc[mask,'installer'] = 'government of tanzania'

    mask = df['installer'].str.contains('danida')
    df.loc[mask,'installer'] = 'danida'

    mask = df['installer'].str.contains('hesawa')
    df.loc[mask,'installer'] = 'hesawa'

    mask = df['installer'].str.contains('rwssp')
    df.loc[mask,'installer'] = 'rwssp'

    mask = df['installer'].str.contains('bank')
    df.loc[mask,'installer'] = 'bank'

    mask = df['installer'].str.contains('ministry')
    df.loc[mask,'installer'] = 'ministry'

    mask = df['installer'].str.contains('germany')
    df.loc[mask,'installer'] = 'germany'

    mask = df['installer'].str.contains('church')
    df.loc[mask,'installer'] = 'church'

    mask = df['installer'].str.contains('unicef')
    df.loc[mask,'installer'] = 'unicef'

    mask = df['installer'].str.contains('council')
    df.loc[mask,'installer'] = 'council'

    mask = df['installer'].str.contains('rural water')
    df.loc[mask,'installer'] = 'rwssp'

    mask = df['installer'].str.contains('water')
    df.loc[mask,'installer'] = 'water'

    mask = df['installer'].str.contains('gov')
    df.loc[mask,'installer'] = 'government of tanzania'

    mask = df['installer'].str.contains('kkk')
    df.loc[mask,'installer'] = 'kkkt'

    installer = df['installer'].value_counts()[:11].index
    mask = df['installer'].isin(installer)
    df.loc[~mask,'installer'] = 'other'

    # Scheme Managment
    df['scheme_management'] = df['scheme_management'].str.lower().fillna('other')
    
    mask = df['scheme_management'].str.contains('none')
    df.loc[mask,'scheme_management'] = 'other'
        
    df.drop(columns=['management','scheme_name','wpt_name','num_private'], inplace=True)
    
    
    # Adding ward Type
    pop_data = pd.read_csv('tza_pop_popn_nbs_baselinedata.xlsx - baselinedata.csv')
    pop_data = pop_data[['Ward_Name', 'ward_type']]
    pop_data.columns = ['ward', 'ward_type']
    df = df.merge(pop_data[['ward', 'ward_type']], on='ward', how='left')
    df['ward_type'] = df['ward_type'].fillna('Unknown')
    
    
    # DATES TO INTS TO OBJECTS
    df['days_const_insp'] = pd.DatetimeIndex(df['date_recorded']) - pd.DatetimeIndex(df['construction_year'])
    df['days_const_insp'] = round(df['days_const_insp'].astype('timedelta64[D]'), 3)
    
    
    df['date_recorded'] = pd.DatetimeIndex(df['date_recorded']).year
    df['construction_year'] = pd.DatetimeIndex(df['construction_year']).year

    df['construction_year'] = df['construction_year'].astype('object')

    
    # HOURS OF USE
    Hours_list = []
    for val in df['population'].fillna(0):
      if val == 0:
        Hours_list.append(0)
      elif val >= 800:
        Hours_list.append(32)
      elif (val >= 400) & (val < 800):
        Hours_list.append(16)
      elif (val >= 200) & (val < 400):
        Hours_list.append(8)
      elif (val >= 100) & (val < 200):
        Hours_list.append(4)
      else:
        Hours_list.append(2)
    df['Hours_per_day'] = Hours_list
    df['Hours_per_day'] = df['Hours_per_day'].replace(0,np.nan)

    df['Used_Hours'] = df['days_const_insp']*df['Hours_per_day']
    
    
    
    df.drop(columns=['region_code', 'ward', 'subvillage', 'region', 'district_code',
                     'date_recorded','Hours_per_day'], inplace=True)
    
    return df

In [491]:
df = wrangle('train_features.csv','train_labels.csv').set_index('id')
X_test = wrangle('test_features.csv')

In [492]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 54816 entries, 454.0 to 23812.0
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   amount_tsh         16554 non-null  float64
 1   funder             54816 non-null  object 
 2   installer          54816 non-null  object 
 3   longitude          53165 non-null  float64
 4   latitude           53165 non-null  float64
 5   basin              54816 non-null  object 
 6   lga                54816 non-null  object 
 7   population         35074 non-null  float64
 8   public_meeting     51477 non-null  object 
 9   scheme_management  54816 non-null  object 
 10  permit             52015 non-null  object 
 11  construction_year  35529 non-null  object 
 12  extraction_type    54816 non-null  object 
 13  management_group   54816 non-null  object 
 14  payment_type       54816 non-null  object 
 15  water_quality      54816 non-null  object 
 16  quantity      

In [493]:
df.describe()

,amount_tsh,longitude,latitude,population,gps_height_sum,days_const_insp,Used_Hours
count,16554.000000,53165.000000,53165.000000,35074.000000,53164.000000,35529.000000,34425.000000
mean,1037.841951,35.218862,-5.953412,281.568712,1047.697451,5628.130935,42533.803980
std,5680.416417,2.626188,2.793186,589.131754,544.706838,4574.556937,64740.775776
min,0.200000,29.607122,-11.649440,1.000000,-63.000000,-2405.000000,-28192.000000
25%,50.000000,33.280987,-8.662783,40.000000,629.300000,1539.000000,6700.000000
50%,200.000000,35.078756,-5.417915,150.000000,1183.100000,4764.000000,19308.000000
75%,600.000000,37.360256,-3.425971,320.000000,1402.425000,9161.000000,47440.000000
max,350000.000000,40.345193,-0.998464,30500.000000,2770.000000,19447.000000,609376.000000


In [494]:
# SPLIT DATA

In [495]:
X = df.drop(columns=['status_group'])
y = df['status_group']

In [496]:
X_train, X_val, y_train, y_val = train_test_split(X,y, random_state = 42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((41112, 23), (13704, 23), (41112,), (13704,))

In [497]:
# BASELINE

In [498]:
y_train.value_counts(normalize=True)

functional                 0.543880
non functional             0.382248
functional needs repair    0.073871
Name: status_group, dtype: float64

In [499]:
baseline_acc = y_train.value_counts(normalize=True).max()
print('Baseline Accuracy Score:', baseline_acc)

Baseline Accuracy Score: 0.5438801323214634


In [500]:
# BUILD MODEL

In [501]:
pipe_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

In [503]:
param_grid = {"randomforestclassifier__max_depth": range(25, 50), #range(19,31,2),
              "randomforestclassifier__n_estimators": range(85,100), #range(104,120,2),
              "randomforestclassifier__max_samples":  [0.3,0.4,0.5],
             "randomforestclassifier__min_samples_split": [5,6,7],
              "randomforestclassifier__min_samples_leaf": [3],
              "randomforestclassifier__bootstrap": [False],
              "randomforestclassifier__criterion":['entropy','gini'],
              "randomforestclassifier__max_features":['log2','sqrt'] #[0.3,0.4,0.5]
             }
model_rfrs = RandomizedSearchCV(
    pipe_rf,
    param_distributions = param_grid,
    n_jobs=-1,
    cv=10,
    verbose=1# status updates
)

# Fit it to the data
model_rfrs.fit(X, y)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(model_rfrs.best_params_))
print("Best score is {}".format(model_rfrs.best_score_))
#Best score is 0.8083923896581758

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Tuned Decision Tree Parameters: {'randomforestclassifier__n_estimators': 86, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__max_samples': 0.3, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__max_depth': 39, 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__bootstrap': False}
Best score is 0.8058413393327658
